# Visualizing EEG Data using MNE


#### Authors: 
- Dr. Rahul Remanan <rahul@moad.computer>
- Brian Parbhu
- Martin Luessi <mluessi@nmr.mgh.harvard.edu>
- Alexandre Gramfort <alexandre.gramfort@telecom-paristech.fr>
- Nicolas P. Rougier (graph code borrowed from his matplotlib gallery)
- Clemens Bruner - (borowed examples from his blog)

#### Install dependencies

In [ ]:
setup = False
set_trace = False
if setup:
    !pip3 install mne
    !pip3 install PyQt5
    !pip3 install PySide2
    !pip3 install scikit-learn
    !pip3 install scipy
    !pip3 install wget

**Import dependencies**

In [ ]:
from scipy.io import loadmat
import wget
from datetime import date
import numpy as np
import ipdb

In [ ]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

In [ ]:
import mne
from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.datasets import sample
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from mne.connectivity import spectral_connectivity
from mne.viz import circular_layout, plot_connectivity_circle

## Part 01 -- Visualization of EEG Channels

**Here's an example of how EEG channels are displayed in the 10-20 system**

> Here we will showcase the different channel positions within the 10-20 system.



In [ ]:
# We need to load our subject data
# If you want to supress the output log of download info use this line
mne.set_log_level("WARNING")
subject = 4
runs = [4, 8 ,12]
raw_fnames = eegbci.load_data(subject, runs)
raw_files = [read_raw_edf(f, preload=True, stim_channel='auto') for f in
             raw_fnames]
raw = concatenate_raws(raw_files)

# Let's print some metadata about our subject data
print(raw)
print(raw.info)

In [ ]:
# You can index metadata about your dataset you want to 
# look via dictionary indexing
print(raw.info["sfreq"])

**Here's how to print out some descriptive aspects of our data**

*  "bads" - List of bad channels (is empty when starting out)
*   "ch_names" - List of the Channel names
*  " chs" - Channel Properties
*  "highpass" - Highpass edge frequencies used during the recordings
* "lowpass" - Lowpass edge frequencies used during the recordings
*   "meas_date" - A timestamp of the recording date







In [ ]:
# Already marked bad channels
print("Bad Channels")
print(raw.info["bads"])

# Channel names
print("Channel names")
print(raw.info["ch_names"])

# Channel properties
print("Channel properties")
print(raw.info["chs"])

# Highpass edge frequencies
print("Highpass frquencies")
print(raw.info["highpass"])

# Lowpass edge frequencies
print("Lowpass frequencies")
print(raw.info["lowpass"])

# Timestamp of the EEG recording- you need to 
# convert the date to a human readable one
print("EEG Timestamp of Recording")
print(date.fromtimestamp(raw.info["meas_date"]))

**Here's an example of how to rename Channels**

In [ ]:
# We are printing out the first 10 channels
raw.info["ch_names"][:10]

In [ ]:
# Here we are going rename the channels because we don't want .. in the names
raw.rename_channels(lambda s: s.strip("."))
print(raw.info["ch_names"][:10])

**Here we are actually going to get a set of montages of coordinate systems that are
relevant to our dataset**



---


*    **Note**: This will also depend on your EEG system as well
* **Also if you're using the 10-10 system you can use the 10-20 system montage as all the channels for the 10-10 system are present**



In [ ]:
# Let's get the montages available to us
mne.channels.get_builtin_montages()

In [ ]:
# Let's pick our montage!
montage = mne.channels.read_montage("standard_1020")
montage.plot()
raw.set_montage(montage)

**Here's also how we determine reference channels in this case**

In [ ]:
# Here we want the reference channel to be 
# the average of all referenced channels
raw.set_eeg_reference("average", projection=False)

**In looking at EEG channels we're also going to be extracting events from channels**

In [ ]:
events = mne.find_events(raw, initial_event=True) 
print(events)
# Values with 2 or 3 are associated with motor imagery onset 
#and 1 is considered to be at rest

In [ ]:
# Way to find annotations automatically with edf files
raw.find_edf_events()

**So lets plot out what our raw data looks like**

In [ ]:
raw.plot()

**So let's see what this looks like when we add our events and annotations**

In [ ]:
raw.plot(n_channels=64, scalings={"eeg": 75e-6}, events=events,
         event_color={1: "green", 2: "blue", 3: "red"})

In [ ]:
# Lets trying plotting in a butterfly style with our events and group them by position
raw.plot(butterfly=True, group_by='position', events=events)

**Now let's plot the power spectrum associated with all 64 channels in our data**

In [ ]:
# Lets do some power spectrum density plots
raw.plot_psd(tmax=np.inf, average=False)

In [ ]:
# Lets do a Power Spectrum Topo Plot
raw.plot_psd_topo(show=True, proj=True)

**So lets do an example of a CSP plot**

In [ ]:
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.
tmin, tmax = -1., 4.
event_id = dict(hands=2, feet=3)
subject = 1
runs = [6, 10, 14]  # motor imagery: hands vs feet

raw_fnames = eegbci.load_data(subject, runs)
raw_files = [read_raw_edf(f, preload=True, stim_channel='auto') for f in
             raw_fnames]
raw = concatenate_raws(raw_files)

In [ ]:
# strip channel names of "." characters
raw.rename_channels(lambda x: x.strip('.'))

In [ ]:
# Apply band-pass filter
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

events = find_events(raw, shortest_event=0, stim_channel='STI 014')

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

In [ ]:
raw.plot()

In [ ]:
# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
labels = epochs.events[:, -1] - 2

In [ ]:
# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

In [ ]:
# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

In [ ]:
# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

In [ ]:
# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

# plot CSP patterns estimated on full data for visualization
csp.fit_transform(epochs_data, labels)

layout = read_layout('EEG1005')
csp.plot_patterns(epochs.info, layout=layout, ch_type='eeg',
                  units='Patterns (AU)', size=1.5)

# Part 02 -- Dealing with Eye Blinks and Bad Channels

**So we're going to show you how to look at Bad EEG channels 
and correct for eye blinks within our EEG Data **

** Lets look at how to deal with eyeblinks first**


In [ ]:
#Lets load up our example dataset
!wget http://bnci-horizon-2020.eu/database/data-sets/001-2014/A01T.mat
mat = loadmat("A01T.mat")

# So our data in this mat file is this really hard to look at dictionary
mat.keys()

# So we need to output everything from the Data key in this dictionary
# Scipy does some weird indexing with mat files
# The things we need are the calibration run and the experiment run from this dataset
# run 3 --calibration
eeg1 = mat["data"][0, 2]["X"][0, 0] * 10e-6 
# run 4 --experiment run
eeg2 = mat["data"][0, 3]["X"][0, 0] * 10e-6 

# We now need to convert these runs into raw eeg objects for mne to use them
# We need to set the number of channels to be 25
# Also we need to identify the sampling frequency as 250 Hz
# We also need to declare the channel types as the first 22 are EEG and
# the last 3 are EOG
info = mne.create_info(25, 250, ch_types=["eeg"] * 22 + ["eog"] * 3)

# We need to to transpose the arrays due to mne expecting channels in rows
# and samples in columns
raw1 = mne.io.RawArray(eeg1.T, info)
raw2 = mne.io.RawArray(eeg2.T, info)

# We need to convert the 3 monopolar EOG channels into 2 bipolar channels
# So to do this we need to multiply monopolar EOG signals with a matrix
# that generates bipolar derivations i.e. EOG1 – EOG2 and EOG3 – EOG2
bip = np.array([[1, -1, 0], [0, -1, 1]])
raw1_eog = bip @ raw1[22:, :][0]
raw2_eog = bip @ raw2[22:, :][0]

raw1_eeg = raw1[:22, :][0]
raw2_eeg = raw2[:22, :][0]


print("Raw1_EOG")
print(raw1_eog)
print("Raw2_EOG")
print(raw2_eog)
print("raw1_eeg")
print(raw1_eeg)
print("raw2_eeg")
print(raw2_eeg)

In [ ]:
# The first method deals with using OLS regression

# Here we're solving for all the dependent and response variables through
# all EEG channels at the same time
# We also have several independent predictor variables coming from our EOG channels
# So we're setting up a multivariate model

# The @ symbol computes the dot product of the matrices
# np.dot can be an alternative to this
betas = np.linalg.inv(raw1_eog @ raw1_eog.T) @ raw1_eog @ raw1_eeg.T
print(betas)

# You can also do it this way as well to compute the regression coefficients
#betas = np.linalg.solve(raw1_eog @ raw1_eog.T, raw1_eog @ raw1_eog.T)

# Lets check the shape of our regresion parameter matrix
# There should be 2 EOG predictors and 22 EEG predictors
#betas.shape()

# So no we need to extract the EOG influence from our EEG data
eeg_corrected = (raw2_eeg.T - raw2_eog.T @ betas).T
raw3 = raw2.copy()
raw3._data[:22, :] = eeg_corrected

In [ ]:
# So now lets look at our data before an eyeblink
raw2.plot(n_channels=25, start=54, duration=4,
          scalings=dict(eeg=250e-6, eog=750e-6))

In [ ]:
# Now lets look at what our data looks like after its been corrected for eye blinks
raw3.plot(n_channels=25, start=54, duration=4,
          scalings=dict(eeg=250e-6, eog=750e-6))

**Now Let's do an example of removing eye blinks in our data with ICA (Independent Components Analysis)**

In [ ]:
# Let's reference our dataset and load our mat file with run 4
eeg_ica = mat["data"][0, 3]["X"][0, 0] * 10e-6

ch_names_ica = ["Fz", "FC3", "FC1", "FCz", "FC2", "FC4", "C5", "C3", "C1", "Cz",
            "C2", "C4", "C6", "CP3", "CP1", "CPz", "CP2", "CP4", "P1", "Pz",
            "P2", "POz", "EOG1", "EOG2", "EOG3"]

# We need to create an info and raw object
info_ica = mne.create_info(ch_names_ica, 250, ch_types=["eeg"] * 22 + ["eog"] * 3)
raw_ica = mne.io.RawArray(eeg_ica.T, info_ica)

raw_ica.set_montage(mne.channels.read_montage("standard_1020"))

In [ ]:
# So ICA does not work in the presance of low frequency drifts
# So here we're going to create a copy of our raw_ica object
# and apply a high-pass filter
raw_tmp = raw_ica.copy()
raw_tmp.filter(1, None, fir_design="firwin")

# We are now going to implement ICA with the extended Infomax
# algorithm and a fixed random state that should make this more reproducible
ica = mne.preprocessing.ICA(method="extended-infomax", random_state=1)
ica.fit(raw_tmp)

**So now we need to check our results and see if they worked**

In [ ]:
# We need to look at all 22 components visually
ica.plot_components(inst=raw_tmp)

In [ ]:
# So ICA001 looks like there might have an eye blink associated with it
# Lets plot out the traces to take a look
ica.plot_sources(inst=raw_tmp)

**So now let's remove the eye blinks associated with our ICA and replot it**

In [ ]:
# So now we're going to create a list of the components we want to exclude
# Then we will apply ica to the corrected eeg signal
ica.exclude = [1]
raw_corrected = raw_ica.copy()
ica.apply(raw_corrected)

In [ ]:
#Lets see what our results looked like before
# Lets plot it out again and see our results
raw_ica.plot(n_channels=25, start=54, duration=4,
         scalings=dict(eeg=250e-6, eog=750e-6))

In [ ]:
# This is what they looked like after
raw_corrected.plot(n_channels=25, start=54, duration=4,
                   scalings=dict(eeg=250e-6, eog=750e-6))

## Part 03 -- MEG sample data visualization

In [ ]:
import os.path as op

In [ ]:
data_path = mne.datasets.sample.data_path()
fname = op.join(data_path, 'MEG', 'sample', 'sample_audvis-ave.fif')
evoked = mne.read_evokeds(fname, baseline=(None, 0), proj=True)

# only left aud
evoked = evoked[0]

lt_chans = [k['ch_name'] for k in evoked.info['chs']
            if k['loc'][0] <= 0 ]

evoked.info['bads'] = []
evoked.plot(exclude=[], show=False)
evoked.info['bads'] += lt_chans
evoked.plot(exclude=[],show=False)
plt.show()

## Part 04 -- Compute source space connectivity and visualize it using a circular graph

This example computes the all-to-all connectivity between 68 regions in source space based on dSPM inverse solutions and a FreeSurfer cortical parcellation. The connectivity is visualized using a circular graph which is ordered based on the locations of the regions.

In [ ]:
data_path = sample.data_path()
subjects_dir = data_path + '/subjects'
fname_inv = data_path + '/MEG/sample/sample_audvis-meg-oct-6-meg-inv.fif'
fname_raw = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
fname_event = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'

In [ ]:
# Load data
inverse_operator = read_inverse_operator(fname_inv)
raw = mne.io.read_raw_fif(fname_raw)
events = mne.read_events(fname_event)

In [ ]:
# Add a bad channel
raw.info['bads'] += ['MEG 2443']

In [ ]:
# Pick MEG channels
picks = mne.pick_types(raw.info, meg=True, eeg=False, stim=False, eog=True,
                       exclude='bads')

In [ ]:
# Define epochs for left-auditory condition
event_id, tmin, tmax = 1, -0.2, 0.5
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,
                    baseline=(None, 0), reject=dict(mag=4e-12, grad=4000e-13,
                                                    eog=150e-6))

In [ ]:
# Compute inverse solution and for each epoch. By using "return_generator=True"
# stcs will be a generator object instead of a list.
snr = 1.0  # use lower SNR for single epochs
lambda2 = 1.0 / snr ** 2
method = "dSPM"  # use dSPM method (could also be MNE or sLORETA)
stcs = apply_inverse_epochs(epochs, inverse_operator, lambda2, method,
                            pick_ori="normal", return_generator=True)

In [ ]:
# Get labels for FreeSurfer 'aparc' cortical parcellation with 34 labels/hemi
labels = mne.read_labels_from_annot('sample', parc='aparc',
                                    subjects_dir=subjects_dir)
label_colors = [label.color for label in labels]

In [ ]:
# Average the source estimates within each label using sign-flips to reduce
# signal cancellations, also here we return a generator
src = inverse_operator['src']
label_ts = mne.extract_label_time_course(stcs, labels, src, mode='mean_flip',
                                         return_generator=True)

In [ ]:
# Now we are ready to compute the connectivity in the alpha band. Notice
# from the status messages, how mne-python: 1) reads an epoch from the raw
# file, 2) applies SSP and baseline correction, 3) computes the inverse to
# obtain a source estimate, 4) averages the source estimate to obtain a
# time series for each label, 5) includes the label time series in the
# connectivity computation, and then moves to the next epoch. This
# behaviour is because we are using generators and allows us to
# compute connectivity in computationally efficient manner where the amount
# of memory (RAM) needed is independent from the number of epochs.
fmin = 8.
fmax = 13.
sfreq = raw.info['sfreq']  # the sampling frequency
con_methods = ['pli', 'wpli2_debiased']
con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
    label_ts, method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)

In [ ]:
# con is a 3D array, get the connectivity for the first (and only) freq. band
# for each method
con_res = dict()
for method, c in zip(con_methods, con):
    con_res[method] = c[:, :, 0]

In [ ]:
# Now, we visualize the connectivity using a circular graph layout

# First, we reorder the labels based on their location in the left hemi
label_names = [label.name for label in labels]

lh_labels = [name for name in label_names if name.endswith('lh')]

In [ ]:
# Get the y-location of the label
label_ypos = list()
for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos.append(ypos)

In [ ]:
# Reorder the labels based on their location
lh_labels = [label for (yp, label) in sorted(zip(label_ypos, lh_labels))]

In [ ]:
# For the right hemi
rh_labels = [label[:-2] + 'rh' for label in lh_labels]

In [ ]:
# Save the plot order and create a circular layout
node_order = list()
node_order.extend(lh_labels[::-1])  # reverse the order
node_order.extend(rh_labels)

node_angles = circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) / 2])

In [ ]:
print (con_res[method])

In [ ]:
# Plot the graph using node colors from the FreeSurfer parcellation. We only
# show the 300 strongest connections.
plot_connectivity_circle(con_res['pli'], label_names, n_lines=300,
                         node_angles=node_angles, node_colors=label_colors,
                         title='All-to-All Connectivity left-Auditory '
                               'Condition (PLI)')
#plt.savefig('circle.png', facecolor='black')
# Plot connectivity for both methods in the same plot
plt.show()

In [ ]:
fig = plt.figure(num=None, figsize=(8, 4), facecolor='black')
no_names = [''] * len(label_names)
for ii, method in enumerate(con_methods):
    if set_trace:
        ipdb.set_trace()
    plot_connectivity_circle(con_res[method], no_names, n_lines=300,
                             node_angles=node_angles, node_colors=label_colors,
                             title=method, padding=0, fontsize_colorbar=6,
                             fig=fig, subplot=(1, 2, ii + 1))

plt.show()